In [38]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser


In [39]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [40]:
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["minedArtists"]

# credentials
TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
title = "Foo Fighters bandstour"
my_nodes = []
my_edges = []


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}

for artist in db.minedArtists.find({"name":"Foo Fighters"}) :
    print artist["name"]
    #print artist
    dates_coords = []
    dates = artist.get("gigs")
    dateini = 0
    for date in dates:
        #print date.get("datetime")
        venue = date.get("venue")
        latitude = venue.get("latitude")
        longitude = venue.get("longitude")
        #print venue
        node = {
            "id" : venue["name"]+str(int(venue["latitude"]))+str(int(venue["longitude"])),
            "name" : venue["name"],
            "lat" : float(venue["latitude"]),
            "lng" : float(venue["longitude"]),
#            "weight" : float(n["weight"]),
#            "start" : n["year_start"],
#            "end" : n["year_stop"]
            }
        if dateini == 0:
            dateini =1
        else:
            #print my_nodes[-1]["data"]["id"]
            edge = {
                "source" : my_nodes[-1]["data"]["id"],
                "target" : node["id"],
#                "weight" : float(e["weight"])
            }
            #print edge
            my_edges.append({ "data" : edge })
 
        my_nodes.append({ "data" : node })
        
    print my_nodes
    print my_edges

        #print "coords",latitude,"/",longitude

Foo Fighters
[{'data': {'lat': 47.6063889, 'lng': -122.3308333, 'id': u'Boat House47-122', 'name': u'Boat House'}}, {'data': {'lat': 45.5236111, 'lng': -122.675, 'id': u'Satyricon45-122', 'name': u'Satyricon'}}, {'data': {'lat': 34.089786, 'lng': -118.268288, 'id': u'Spaceland34-118', 'name': u'Spaceland'}}, {'data': {'lat': 38.9716667, 'lng': -95.235, 'id': u'Bottleneck38-95', 'name': u'Bottleneck'}}, {'data': {'lat': 38.63172, 'lng': -90.182968, 'id': u'Mississippi Nights38-90', 'name': u'Mississippi Nights'}}, {'data': {'lat': 39.129952, 'lng': -84.509613, 'id': u"Bogart's39-84", 'name': u"Bogart's"}}, {'data': {'lat': 33.75099, 'lng': -84.390272, 'id': u'Masquerade33-84', 'name': u'Masquerade'}}, {'data': {'lat': 33.958224, 'lng': -83.379945, 'id': u'40 Watt Club33-83', 'name': u'40 Watt Club'}}, {'data': {'lat': 35.910615, 'lng': -79.068873, 'id': u"Cat's Cradle35-79", 'name': u"Cat's Cradle"}}, {'data': {'lat': 38.891312, 'lng': -77.033653, 'id': u'Black Cat38-77', 'name': u'Blac

In [34]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)

create_topogram(title, my_nodes, my_edges)

ERROR:topogram-client:403 Unauthorized request
ERROR:topogram-client:500 - Error : A topogram with the same name already exists


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'UpI_nqpGKfQpS3ifYtzeU3Bqqs_ayNOpI7IoOhaMybp', u'userId': u'DaofbRenqYpqqNmtz'}}
Creating topogram 'Geo-time network'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'Geo-time network', u'userId': u'DaofbRenqYpqqNmtz', u'_id': u'R2euSusu5Ydrmigx6', u'slug': u'geo-time-network', u'createdAt': u'2017-10-12T09:57:24.966Z'}, u'statusCode': 200}
topogram ID : R2euSusu5Ydrmigx6
941 nodes deleted
940 edges deleted
{u'status': u'success', 'status_code': 200, u'data': [u'DpSkLREiQNSMkrdyW', u'oM9SKmJaHhv4u8LMN', u'eCowzbANyz8HLQJ8u', u'2Ww3fqSAPnPS2oxvH', u'rAtpDgM7ufx3iGHtS', u'a6J8F69isCLTZbcsn', u'WvAcNZFQnxY7qQ9Nt', u'ywjkR5fxm4pFEGbuy', u'7PWTjDeMQ9fFCeFLS', u'hHzwsBBMcmsMSpwoN', u'9WAouFfGixEMnyQ9q', u'pegGZiLessdpbCiTR', u'spFR38wco5AwkiMm6', u'vDwvuHn5PZ6qZbGDK', u'bvLoPmpfPqpacZPYy', u'DvJcDiDf8YpLwb8X2', u'3qTLETXYfMRt

940 edges created.
done. Topogram has been updated. Check it at https://app.topogram.io/topograms/R2euSusu5Ydrmigx6


In [ ]:
    #print artist
    

        dates_coords.append( [latitude,longitude])
    #print "datec",date_coords
    datapushed={}
    datapushed["dates_coords_lat_long"]=dates_coords
    datapushed["id"]=artist["_id"]
    datapushed["slug"]=artist["slug"]
    print datapushed
    dict_coords[artist["slug"]]=datapushed
print dict_coords



with open('data/test_data_nodes.csv') as f :
    reader = DictReader(f)
    for n in reader :
        node = {
            "id" : n["id"],
            "name" : n["name"],
            "lat" : float(n["lat"]),
            "lng" : float(n["lng"]),
            "weight" : float(n["weight"]),
            "start" : n["year_start"],
            "end" : n["year_stop"]
            }
        my_nodes.append({ "data" : node })

print my_nodes

with open('data/test_data_edges.csv') as f :
    reader = DictReader(f)
    for e in reader :
        edge = {
            "source" : e["source"],
            "target" : e["target"],
            "weight" : float(e["weight"])
        }
        my_edges.append({ "data" : edge })

# print my_edges

def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)

    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)

# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)

create_topogram("Geo-time network", my_nodes, my_edges)